In [1]:
# 패키지 로딩하기
import re
import numpy as np
import tensorflow as tf
import os
import glob
import sklearn.model_selection

In [2]:
# 데이터 불러오기
txt_file_path = os.getenv("HOME") + "/aiffel/lyricist/data/lyrics/*" # ~/aiffel/lyricist/data/lyrics 폴더 안의 모든 파일 선택
txt_list      = glob.glob(txt_file_path)                             # 파일 경로 안의 모든 파일 이름 리스트형태로 반환

In [3]:
raw_corpus = []                     # 미리 빈 리스트를 만들고

for txt_file in txt_list:           # txt_list을 하나씩 가져와서 txt_file이라고 명명하고
    with open(txt_file, "r") as f:  # txt_file을 읽기전용으로 열고 f라고 명명하고
        raw = f.read().splitlines() # f을 한줄씩 읽어와서
        raw_corpus.extend(raw)      # 미리 만들어 놓은 raw_corpus에 넣는다. (iterable로 넣기위해 extend 사용)

In [4]:
print("전체 문장 갯수 :", len(raw_corpus))
print("예시 문장 5개 :", raw_corpus[:5])

# 연극 대사처럼 따로 빼거나 없애야 하는 특수문자는 보이지 않음

전체 문장 갯수 : 187088
예시 문장 5개 : ['The first words that come out', 'And I can see this song will be about you', "I can't believe that I can breathe without you", 'But all I need to do is carry on', 'The next line I write down']


In [5]:
# 데이터 정제하기 (데이터 전처리)
# 토큰화 했을 때 토큰 갯수가 15개를 넘어가는 문장은 학습데이터에서 제외함. (노드에서 시킴)
def preprocess_sentence(sentence):      # 문장을 정제하는 함수를 만들 것임
    sentence = sentence.lower().strip() # 먼저 들어온 문장을 소문자로 바꾸고 양쪽 공백이 있으면 제거함
    
    # 문장에서 처리해줘야 할 기호나 패턴을 처리
    # 실습 데이터는 그대로 써도 될 것같지만 혹시 몰라서 노드에 있는 경우의 수를 그대로 씀
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)   # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)          # 공백 패턴을 만나면 공백 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 알파벳(대,소문자), ?.!,¿패턴을 제외한 모든 문자(공백 포함)를 공백 1개로 치환
    
    sentence = sentence.strip()                          # 한번 더 양쪽 공백 제거
    sentence = "<start> " + sentence + " <end>"          # sentence 양쪽에 시작과 끝을 알리는 단어를 붙여줌
    
    return sentence

In [6]:
# 함수가 잘 만들어졌는지 확인하기
print(preprocess_sentence("Test : It's @#%^$#%$ sample     text"))

# 공백, 특수문자 전부 잘 처리됨.

<start> test it s sample text <end>


In [7]:
corpus = [] # 빈 리스트를 미리 만들고

for sentence in raw_corpus:                       # 기존의 텍스트 데이터를 한개씩 가져와서
    if len(sentence) == 0: continue               # 아무것도 없는 것은 제외하고
    temp_sentence = preprocess_sentence(sentence) # 정제하는 함수를 거친 뒤
    if len(temp_sentence.split()) > 15: continue  # 토큰의 갯수가 15개를 넘어가는 문장은 데이터에서 제외하고
    corpus.append(temp_sentence)                  # 리스트에 추가한다.

corpus[:9] # 잘 뽑혔는지 10개만 출력하여 확인하기

# 잘 작동함.

['<start> the first words that come out <end>',
 '<start> and i can see this song will be about you <end>',
 '<start> i can t believe that i can breathe without you <end>',
 '<start> but all i need to do is carry on <end>',
 '<start> the next line i write down <end>',
 '<start> and there s a tear that falls between the pages <end>',
 '<start> i know that pain s supposed to heal in stages <end>',
 '<start> i could throw it in the river and watch it sink in slowly <end>',
 '<start> tie the pages to a plane and send it to the moon <end>']

In [8]:
print("corpus 갯수 :", len(corpus)) # 156200이 나와야함

corpus 갯수 : 156227


In [9]:
# 정제한 데이터를 텐서(벡터화)로 만들기
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 12000,   # 전체 단어의 개수 - 하이퍼파라미터
                                                      filters   = " ",     # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
                                                      oov_token = "<unk>") # 사전에 없었던 단어는 어떤 토큰으로 대체할지 정한다.
    tokenizer.fit_on_texts(corpus) # corpus로부터 tokenizer가 사전을 자동 구축한다.
    
    tensor = tokenizer.texts_to_sequences(corpus) # 구축한 사전으로부터 corpus를 해석해 tensor로 변환한다.
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공한다. 
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = "post")
    
    return tensor, tokenizer

In [10]:
tensor, tokenizer = tokenize(corpus)
print(tensor.shape)

(156227, 15)


In [11]:
# 156227개의 문장이 있고, 문장의 토큰 최대길이는 15이다.
print(tensor[:5, ])

[[  2   6 247 434  15  68  57   3   0   0   0   0   0   0   0]
 [  2   8   4  35  63  41 357  84  27 112   7   3   0   0   0]
 [  2   4  35  16 218  15   4  35 768 257   7   3   0   0   0]
 [  2  33  25   4  92  10  48  26 832  18   3   0   0   0   0]
 [  2   6 330 442   4 760  58   3   0   0   0   0   0   0   0]]


In [12]:
# 평가 데이터셋 분리
source_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
target_input = tensor[:, 1:]  # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(source_input[0])
print(target_input[0])

[  2   6 247 434  15  68  57   3   0   0   0   0   0   0]
[  6 247 434  15  68  57   3   0   0   0   0   0   0   0]


In [13]:
enc_train, enc_val, dec_train, dec_val = sklearn.model_selection.train_test_split(source_input, target_input, test_size = 0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


In [14]:
print(enc_val.shape)
print(dec_val.shape)

(31246, 14)
(31246, 14)


In [15]:
buffer_size     = len(source_input)
batch_size      = 256
steps_per_epoch = buffer_size // batch_size

vocab_size = tokenizer.num_words + 1 # tokenizer가 구축한 단어사전 내 12000개 + 여기에 포함되지 않은 0 : <pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((source_input, target_input)).shuffle(buffer_size)
dataset = dataset.batch(batch_size, drop_remainder = True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [16]:
# 모델 만들기
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1     = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2     = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear    = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [17]:
model = TextGenerator(vocab_size = vocab_size, embedding_size = 256, hidden_size = 1024)

In [18]:
for source_sample, target_sample in dataset.take(1): break
model(source_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.22726451e-05,  8.87333663e-05, -2.94235474e-06, ...,
          1.29466309e-04,  8.79908330e-05,  1.04970401e-04],
        [-4.37639355e-05,  5.63477515e-05,  5.54324834e-05, ...,
          4.21560602e-04, -2.53890292e-04,  3.04111076e-04],
        [-4.68668040e-05, -2.25214259e-04,  9.29941962e-05, ...,
          4.02526523e-04, -1.71120351e-04,  5.04019845e-04],
        ...,
        [ 6.93021109e-04, -6.56319200e-04,  7.33196095e-04, ...,
          1.05014641e-03, -4.54097637e-04,  1.29983146e-05],
        [ 1.00782781e-03, -7.40238174e-04,  7.01142184e-04, ...,
          9.62601334e-04,  1.32387955e-04,  2.07413577e-05],
        [ 1.33983837e-03, -8.72452685e-04,  6.37167250e-04, ...,
          8.62627872e-04,  7.60719879e-04, -2.69910151e-06]],

       [[ 1.22726451e-05,  8.87333663e-05, -2.94235474e-06, ...,
          1.29466309e-04,  8.79908330e-05,  1.04970401e-04],
        [-1.08053373e-05, -2.13812789e-04,  1

In [19]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [20]:
optimizer = tf.keras.optimizers.Adam()
loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = "none")

model.compile(loss = loss, optimizer = optimizer)
history = model.fit(dataset, epochs = 10)

Epoch 1/10
610/610 [==============================] - 74s 122ms/step - loss: 3.4292
Epoch 2/10
610/610 [==============================] - 75s 124ms/step - loss: 2.9581
Epoch 3/10
610/610 [==============================] - 76s 124ms/step - loss: 2.7871
Epoch 4/10
610/610 [==============================] - 76s 124ms/step - loss: 2.6577
Epoch 5/10
610/610 [==============================] - 76s 124ms/step - loss: 2.5460
Epoch 6/10
610/610 [==============================] - 76s 125ms/step - loss: 2.4453
Epoch 7/10
610/610 [==============================] - 76s 125ms/step - loss: 2.3531
Epoch 8/10
610/610 [==============================] - 76s 125ms/step - loss: 2.2681
Epoch 9/10
610/610 [==============================] - 76s 125ms/step - loss: 2.1891
Epoch 10/10
610/610 [==============================] - 77s 126ms/step - loss: 2.1138


In [21]:
def generate_text(model, tokenizer, init_sentence = "<start>", max_len = 20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input  = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype = tf.int64)
    end_token   = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict      = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis = 0)], axis = -1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [25]:
generate_text(model, tokenizer, init_sentence = "<start> i love", max_len = 20)

'<start> i love you , i love you <end> '

In [ ]:
# 회고록
# 여기서 사용한 def 함수는 노드를 인용하여 만들었음 (보고 그대로 쓰고나, 일부 바꾸거나)
# 노드를 참고하여 데이터를 가공하고 나누는데 train 데이터 갯수가 노드대로 안 나와서 조원과 상의하면서 어디가 잘못됬는지 한참 찾음.
# 나중에 퍼실님께서 오셔서 신경쓸 필요 없다고 하여 넘어가도 되는걸 깨달음....
# 노드에 나온것처럼 모델을 class로 만들고 학습을 시작함. (학습시간도 엄청 걸림)
# 그리고 프로젝트에서 준 단어(i love)를 모델에게 제시하고 문장을 만들라고 했더니 너무 쉬운 문장이 나와서 이게 맞는건가 싶음.
# loss는 그래도 2.2 이하로 내려감.